### Código para descargar y consolidar datos de Quién es quién en los precios de Profeco

La información es de 2015 a 2022

In [1]:
#Librerías
import pandas as pd
import requests
import os
import patoolib
import datetime
import locale
import re

# Locale para Mex
locale.setlocale(locale.LC_ALL, 'es_MX')

'es_MX'

In [2]:
#Cambiar directorio de trabajo
try:
    os.chdir("C:/Users/claudio.pacheco/Documents/")
except:
    os.chdir("C:/Users/claud/Documents/")
#Crear carpeta. Si ya existe, no la crea
os.makedirs("profeco", exist_ok=True)

In [3]:
#Lista de urls
urls=["https://datos.profeco.gob.mx/datos_abiertos/file.php?t=976288eab7d5db41e8f99258b2f93a6b"
      ]

In [4]:
# Define the destination folder for the downloaded files
folder_path = "profeco"

for url in urls:
    # Define the file name and path
    file_name = "temp.rar"
    file_path = os.path.join(folder_path, file_name)
    
    try:
        # Download the file
        print(f"Descargando {url}...")
        response = requests.get(url)
        
        # Check if the response status is OK (200)
        response.raise_for_status()

        # Save the file to the destination folder
        with open(file_path, 'wb') as f:
            f.write(response.content)
        
        print(f"Descarga completada: {url}")
        
        # Extract the files from the RAR archive
        print(f"Extrayendo archivos...")
        patoolib.extract_archive(file_path, outdir=folder_path)
        print("Extracción completada")
        # Remove the RAR file
        os.remove(file_path)
        
    except requests.RequestException as e:
        print(f"Error descargando {url}. Error: {e}")
    except Exception as e:
        print(f"Error descargando {url}. Error: {e}")


Descargando https://datos.profeco.gob.mx/datos_abiertos/file.php?t=976288eab7d5db41e8f99258b2f93a6b...
Descarga completada: https://datos.profeco.gob.mx/datos_abiertos/file.php?t=976288eab7d5db41e8f99258b2f93a6b
Extrayendo archivos...
patool: Extracting profeco\temp.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- C:\Users\claud\Documents\profeco\temp.rar
patool:     with cwd=profeco
patool: ... profeco\temp.rar extracted to `profeco'.
Extracción completada


In [5]:
#Concatenar archivos de las carpetas en un solo dataframe
#Lista de carpetas
carpetas = os.listdir("profeco")
archivos = [os.listdir("profeco/"+carpeta) for carpeta in carpetas]

In [6]:
archivos

[['01-2023_01.csv',
  '01-2023_02.csv',
  '02-2023_01.csv',
  '02-2023_02.csv',
  '03-2023_01.csv',
  '03-2023_02.csv',
  '04-2023_01.csv',
  '04-2023_02.csv',
  '05-2023_01.csv',
  '05-2023_02.csv',
  '06-2023_01.csv',
  '06-2023_02.csv',
  '07-2023_01.csv',
  '07-2023_02.csv',
  '08-2023_01.csv',
  '08-2023_02.csv',
  '09-2023_01.csv',
  '09-2023_02.csv']]

In [7]:
# Initialize an empty list to hold dataframes
dfs = []

for i, folder in enumerate(carpetas):
    for j, file in enumerate(archivos[i]):
        # Construct the full file path
        file_path = "profeco/" + folder + "/" + file
        
        # Read the CSV into a dataframe
        temp_df = pd.read_csv(file_path, encoding="utf-8", names=[
            "producto", "presentacion", "marca", "categoria", "catalogo", "precio",
            "fecharegistro", "cadenacomercial", "giro", "nombrecomercial", "direccion",
            "estado", "municipio", "latitud", "longitud"
        ])
        
        # Append the dataframe to the list
        dfs.append(temp_df)
        print(f"Concatenando archivo {j+1} de {len(archivos[i])}")

# Concatenate all dataframes
df = pd.concat(dfs, ignore_index=True)
#Eliminar acentos
df["producto"] = df["producto"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
# Filter data by catalogo
print("Filtrando datos: Dejar solo productos PACIC")
pacic = df[df["catalogo"] == "PACIC"]
#Todo en minúsculas
print("Proceso terminado")

Concatenando archivo 1 de 18
Concatenando archivo 2 de 18
Concatenando archivo 3 de 18
Concatenando archivo 4 de 18
Concatenando archivo 5 de 18
Concatenando archivo 6 de 18
Concatenando archivo 7 de 18
Concatenando archivo 8 de 18
Concatenando archivo 9 de 18
Concatenando archivo 10 de 18
Concatenando archivo 11 de 18
Concatenando archivo 12 de 18
Concatenando archivo 13 de 18
Concatenando archivo 14 de 18
Concatenando archivo 15 de 18
Concatenando archivo 16 de 18
Concatenando archivo 17 de 18
Concatenando archivo 18 de 18
Filtrando datos: Dejar solo productos PACIC
Proceso terminado


In [8]:
#Se crean funciones para el análisis de datos
def examinar(df):
    #Examinar productos únicos
    print("Productos únicos en la tabla: ")
    print(df["producto"].unique())
    print("Número de productos únicos: ", len(df["producto"].unique()))
    print("Número de registros: ", format(len(df), ",d"))

def filtro_pacic(df):
    filtro = ["ACEITE", "ARROZ", "ATUN", "AZUCAR", "CARNE RES","CEBOLLA", "CHILE FRESCO",
              "CARNE CERDO", "FRIJOL","HUEVO", "JABON DE TOCADOR", "JITOMATE", "LECHE ULTRAPASTEURIZADA",
                "LIMON", "MANZANA", "NARANJA", "PAN DE CAJA", "PAPA", "PAPEL HIGIENICO", "PASTA PARA SOPA",
                "CARNE POLLO", "SARDINA", "TORTILLA DE MAIZ", "ZANAHORIA"]
    return df[df["producto"].isin(filtro)]
    
def filtro_supermercado(df):
    filtro = ['MEGA SORIANA', 'WAL-MART','BODEGA AURRERA', 'CHEDRAUI',
              'HIPERMERCADO SORIANA', 'SUPERMERCADOS LEY','SORIANA SUPER', 'CASA LEY', 'SORIANA EXPRESS',
              'MERCADO SORIANA','LA COMER', 'SUMESA', 'WAL-MART EXPRESS', 'BODEGA AURRERA EXPRESS',
              'CHEDRAUI SELECTO', 'SORIANA', 'SUPER CHEDRAUI', 'CITY MARKET', 'FRESKO LA COMER','LEY (AUTOSERVICIO)',
              'LEY EXPRESS']

    # Filter
    df_filtered = df[(df['giro'] == 'SUPERMERCADO / TIENDA DE AUTOSERVICIO') & (df['cadenacomercial'].isin(filtro))]

    # Create "grupo" column based on "cadenacomercial"
    def categorize_chain(chain):
        if chain in ["WAL-MART", "WAL-MART EXPRESS", "BODEGA AURRERA", "BODEGA AURRERA EXPRESS"]:
            return "WALMART"
        elif chain in ["SORIANA SUPER", "MEGA SORIANA", "HIPERMERCADO SORIANA", "MERCADO SORIANA", "SORIANA EXPRESS", "SORIANA", "LA COMER", "SUMESA", "CITY MARKET", "FRESKO LA COMER"]:
            return "SORIANA"
        elif chain in ["CHEDRAUI", "CHEDRAUI SELECTO", "SUPER CHEDRAUI"]:
            return "CHEDRAUI"
        elif chain in ["SUPERMERCADOS LEY", "CASA LEY", "LEY (AUTOSERVICIO)", "LEY EXPRESS"]:
            return "LEY"
        else:
            return chain

    df_filtered["grupo"] = df_filtered["cadenacomercial"].apply(categorize_chain)
    return df_filtered

In [9]:
examinar(pacic)

Productos únicos en la tabla: 
['ACEITE' 'ARROZ' 'ATUN' 'AZUCAR' 'CARNE CERDO' 'CARNE POLLO' 'CARNE RES'
 'CEBOLLA' 'CHILE FRESCO' 'FRIJOL' 'JABON DE TOCADOR' 'JITOMATE'
 'LECHE ULTRAPASTEURIZADA' 'LIMON' 'MANZANA' 'PAN DE CAJA'
 'PAPEL HIGIENICO' 'PASTA PARA SOPA' 'PLATANO' 'SARDINA'
 'TORTILLA DE MAIZ' 'ZANAHORIA' 'HUEVO' 'PAPA' 'LECHE PASTEURIZADA'
 'ESPINACAS' 'JAMON' 'LECHUGA' 'LENTEJA' 'MOJARRA' 'PURE DE TOMATE'
 'SERVILLETAS DE PAPEL BLANCO' 'TOALLA FEMENINA' 'PINA' 'TILAPIA']
Número de productos únicos:  35
Número de registros:  530,083


In [10]:
examinar(df)

Productos únicos en la tabla: 
['A.S.COR' 'ACEITE' 'ACEITE DE OLIVA' 'ACEITUNA' 'ACELGA' 'ACICLOVIR'
 'ACIDO FOLICO' 'ACLIMAFEL' 'ACONDICIONADOR Y ENJUAGUE' 'ADALAT OROS'
 'ADEROGYL 15' 'ADIMOD' 'AFUMIX' 'AFUNGIL' 'AGRIFEN' 'AGUA CON GAS'
 'AGUA SIN GAS' 'AGUACATE' 'ALCOHOL' 'ALEVIAN DUO' 'ALGODON'
 'ALIMENTO PREPARADO PARA NINOS' 'ALIMENTOS PREPADOS' 'ALKA-SELTZER'
 'ALTRULINE' 'AMARYL M' 'AMBROXOL DEXTROMETORFANO' 'AMLODIPINO'
 'AMOXICILINA' 'AMOXICLAV. BID' 'AMPICILINA' 'AMPLIRON DUO' 'ANGELIQ A'
 'ANTIFLU-DES' 'ANTIFLU-DES JR.' 'APROVEL' 'ARAVA' 'ARCOXIA' 'ARROZ'
 'ARTRIDOL' 'ASENLIX' 'ASPIRADORAS' 'ASPIRINA' 'ASPIRINA PROTECT'
 'ATACAND' 'ATACAND PLUS' 'ATORVASTATINA' 'ATUN' 'AUGMENTIN 12 H.'
 'AUTOPISTAS' 'AVAMYS 2A' 'AVELOX 7D' 'AVENA' 'AVODART' 'AZITROMICINA'
 'AZUCAR' 'BACTRIM F' 'BARRA DE SONIDO' 'BATIDORAS' 'BEBIDAS HIDRATANTES'
 'BEDOYECTA' 'BEDOYECTA-TRI' 'BENZONATATO' 'BETABEL'
 'BETAMETASONA/ CLOTRIMAZOL/ GENTAMICINA' 'BEXIDENT ENCIAS' 'BEZAFIBRATO'
 'BIOMETRIX' 'BIOMICS

In [11]:
pacic2=filtro_pacic(df)
print("Número de registros:", format(len(pacic2), ",d"))
print("Productos considerados en tabla pacic que no se encuentran en la pacic2: ")
set(pacic["producto"].unique()) - set(pacic2["producto"].unique())

Número de registros: 1,847,036
Productos considerados en tabla pacic que no se encuentran en la pacic2: 


{'ESPINACAS',
 'JAMON',
 'LECHE PASTEURIZADA',
 'LECHUGA',
 'LENTEJA',
 'MOJARRA',
 'PINA',
 'PLATANO',
 'PURE DE TOMATE',
 'SERVILLETAS DE PAPEL BLANCO',
 'TILAPIA',
 'TOALLA FEMENINA'}

In [12]:
#Filtro de supermercados
pacic2=filtro_supermercado(pacic2)
examinar(pacic2)

C:\Users\claud\AppData\Local\Temp\ipykernel_35092\2972760854.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["grupo"] = df_filtered["cadenacomercial"].apply(categorize_chain)


Productos únicos en la tabla: 
['ACEITE' 'ARROZ' 'ATUN' 'AZUCAR' 'CARNE CERDO' 'CARNE POLLO' 'CARNE RES'
 'CEBOLLA' 'CHILE FRESCO' 'FRIJOL' 'HUEVO' 'JABON DE TOCADOR' 'JITOMATE'
 'LECHE ULTRAPASTEURIZADA' 'LIMON' 'MANZANA' 'NARANJA' 'PAN DE CAJA'
 'PAPEL HIGIENICO' 'PASTA PARA SOPA' 'SARDINA' 'TORTILLA DE MAIZ'
 'ZANAHORIA' 'PAPA']
Número de productos únicos:  24
Número de registros:  1,564,883


In [13]:
#Obtener el precio de la canasta por grupo
pacic2

,producto,presentacion,marca,categoria,catalogo,precio,fecharegistro,cadenacomercial,giro,nombrecomercial,direccion,estado,municipio,latitud,longitud,grupo
1,ACEITE,BOTELLA 1 LT. VEGETAL,1-2-3,ACEITES Y GRASAS VEG. COMESTIBLES,BASICOS,56.0,2023-01-02,MEGA SORIANA,SUPERMERCADO / TIENDA DE AUTOSERVICIO,MEGA SORIANA SUCURSAL AGUASCALIENTES,"AGUASCALIENTES NTE. 802, ESQ. BLVD. A ZACATECA...",AGUASCALIENTES,AGUASCALIENTES,21.916001,-102.290779,SORIANA
2,ACEITE,BOTELLA 850 ML. VEGETAL,AVE,ACEITES Y GRASAS VEG. COMESTIBLES,PACIC,39.0,2023-01-02,MEGA SORIANA,SUPERMERCADO / TIENDA DE AUTOSERVICIO,MEGA SORIANA SUCURSAL AGUASCALIENTES,"AGUASCALIENTES NTE. 802, ESQ. BLVD. A ZACATECA...",AGUASCALIENTES,AGUASCALIENTES,21.916001,-102.290779,SORIANA
3,ACEITE,BOTELLA 946 ML. CANOLA,CANOIL,ACEITES Y GRASAS VEG. COMESTIBLES,BASICOS,50.9,2023-01-02,MEGA SORIANA,SUPERMERCADO / TIENDA DE AUTOSERVICIO,MEGA SORIANA SUCURSAL AGUASCALIENTES,"AGUASCALIENTES NTE. 802, ESQ. BLVD. A ZACATECA...",AGUASCALIENTES,AGUASCALIENTES,21.916001,-102.290779,SORIANA
4,ACEITE,BOTELLA 840 ML. CANOLA,CAPULLO,ACEITES Y GRASAS VEG. COMESTIBLES,BASICOS,73.9,2023-01-02,MEGA SORIANA,SUPERMERCADO / TIENDA DE AUTOSERVICIO,MEGA SORIANA SUCURSAL AGUASCALIENTES,"AGUASCALIENTES NTE. 802, ESQ. BLVD. A ZACATECA...",AGUASCALIENTES,AGUASCALIENTES,21.916001,-102.290779,SORIANA
5,ACEITE,BOTELLA 900 ML. VEGETAL,GRAN TRADICIÓN,ACEITES Y GRASAS VEG. COMESTIBLES,BASICOS,45.5,2023-01-02,MEGA SORIANA,SUPERMERCADO / TIENDA DE AUTOSERVICIO,MEGA SORIANA SUCURSAL AGUASCALIENTES,"AGUASCALIENTES NTE. 802, ESQ. BLVD. A ZACATECA...",AGUASCALIENTES,AGUASCALIENTES,21.916001,-102.290779,SORIANA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11380584,SARDINA,LATA 425 GR. EN TOMATE,YAVAROS,PESCADOS Y MARISCOS EN CONSERVA,BASICOS,39.5,2023-09-29,BODEGA AURRERA,SUPERMERCADO / TIENDA DE AUTOSERVICIO,BODEGA AURRERA SUCURSAL SALERO 4042,"GARCIA SALINAS S/N, ESQ. CALLEJON DEL CHAVERO,...",ZACATECAS,ZACATECAS,22.752156,-102.527606,WALMART
11380648,TORTILLA DE MAIZ,1 KG. GRANEL,S/M,TORTILLAS Y DERIVADOS DEL MAIZ,BASICOS,12.8,2023-09-29,BODEGA AURRERA,SUPERMERCADO / TIENDA DE AUTOSERVICIO,BODEGA AURRERA SUCURSAL SALERO 4042,"GARCIA SALINAS S/N, ESQ. CALLEJON DEL CHAVERO,...",ZACATECAS,ZACATECAS,22.752156,-102.527606,WALMART
11380649,TORTILLA DE MAIZ,1 KG. GRANEL,S/M,TORTILLAS Y DERIVADOS DEL MAIZ,PACIC,12.8,2023-09-29,BODEGA AURRERA,SUPERMERCADO / TIENDA DE AUTOSERVICIO,BODEGA AURRERA SUCURSAL SALERO 4042,"GARCIA SALINAS S/N, ESQ. CALLEJON DEL CHAVERO,...",ZACATECAS,ZACATECAS,22.752156,-102.527606,WALMART
11380674,ZANAHORIA,1 KG. MEDIANA,S/M,HORTALIZAS FRESCAS,FRUTAS Y LEGUMBRES,25.9,2023-09-29,BODEGA AURRERA,SUPERMERCADO / TIENDA DE AUTOSERVICIO,BODEGA AURRERA SUCURSAL SALERO 4042,"GARCIA SALINAS S/N, ESQ. CALLEJON DEL CHAVERO,...",ZACATECAS,ZACATECAS,22.752156,-102.527606,WALMART


In [14]:
#Obtener precio promedio de la canasta por grupo
pacic2.groupby("grupo")["precio"].mean()

grupo
CHEDRAUI    50.350619
LEY         49.335412
SORIANA     48.387395
WALMART     52.676327
Name: precio, dtype: float64